In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SQL_Data_Frame').getOrCreate()

24/12/05 18:30:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# Show csv file data

In [4]:
%%bash
head -10 ./data/persons.csv

name,age,gender,salary
John Doe,30,Male,50000
Jane Smith,25,Female,45000
David Johnson,35,Male,60000
Emily Davis,28,Female,52000
Michael Wilson,40,Male,75000
Sarah Brown,32,Female,58000
Robert Lee,29,Male,51000
Lisa Garcia,27,Female,49000
James Martinez,38,Male,70000


### Load Data into DataFrame

In [6]:
df_path = './data/persons.csv'

In [7]:
df = spark.read.csv(df_path, header=True, inferSchema=True)

In [8]:
# Dispaly Schema of the Dataframe

In [9]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



In [10]:
# Show the intial DataFrame

In [11]:
print('Inatial DataFrame')
df.show()

Inatial DataFrame
+------------------+---+------+------+
|              name|age|gender|salary|
+------------------+---+------+------+
|          John Doe| 30|  Male| 50000|
|        Jane Smith| 25|Female| 45000|
|     David Johnson| 35|  Male| 60000|
|       Emily Davis| 28|Female| 52000|
|    Michael Wilson| 40|  Male| 75000|
|       Sarah Brown| 32|Female| 58000|
|        Robert Lee| 29|  Male| 51000|
|       Lisa Garcia| 27|Female| 49000|
|    James Martinez| 38|  Male| 70000|
|Jennifer Rodriguez| 26|Female| 47000|
|  William Anderson| 33|  Male| 62000|
|   Karen Hernandez| 31|Female| 55000|
|Christopher Taylor| 37|  Male| 69000|
|     Mary Gonzalez| 24|Female| 44000|
|     Matthew Davis| 36|  Male| 67000|
|    Patricia White| 29|Female| 50000|
|     Daniel Miller| 34|  Male| 64000|
| Elizabeth Jackson| 30|Female| 52000|
|     Joseph Harris| 28|  Male| 53000|
|      Linda Martin| 39|Female| 71000|
+------------------+---+------+------+



### Register the DataFrame as Temp Table

In [16]:
df.createOrReplaceTempView('persons')

### Perform SQL like Quries

In [15]:
rslt = spark.sql("SELECT * FROM persons WHERE age > 30")
rslt.show()

+------------------+---+------+------+
|              name|age|gender|salary|
+------------------+---+------+------+
|     David Johnson| 35|  Male| 60000|
|    Michael Wilson| 40|  Male| 75000|
|       Sarah Brown| 32|Female| 58000|
|    James Martinez| 38|  Male| 70000|
|  William Anderson| 33|  Male| 62000|
|   Karen Hernandez| 31|Female| 55000|
|Christopher Taylor| 37|  Male| 69000|
|     Matthew Davis| 36|  Male| 67000|
|     Daniel Miller| 34|  Male| 64000|
|      Linda Martin| 39|Female| 71000|
+------------------+---+------+------+



In [18]:
av_salary_gender = spark.sql('SELECT gender, AVG(salary) AS avg_salary FROM persons GROUP BY gender')
av_salary_gender.show()

+------+----------+
|gender|avg_salary|
+------+----------+
|Female|   52300.0|
|  Male|   62100.0|
+------+----------+



### Creating and managing temporary views.

In [19]:
# Create a temporary view
df.createOrReplaceTempView("people")

In [21]:
rslts = spark.sql('SELECT * FROM people WHERE age >33')
rslts.show()

+------------------+---+------+------+
|              name|age|gender|salary|
+------------------+---+------+------+
|     David Johnson| 35|  Male| 60000|
|    Michael Wilson| 40|  Male| 75000|
|    James Martinez| 38|  Male| 70000|
|Christopher Taylor| 37|  Male| 69000|
|     Matthew Davis| 36|  Male| 67000|
|     Daniel Miller| 34|  Male| 64000|
|      Linda Martin| 39|Female| 71000|
+------------------+---+------+------+



In [26]:
# Check if a temporay view exists
view_exists = spark.catalog.tableExists('people')
view_exists

True

### Drop a temp view

In [29]:
# Drop a temporary view
spark.catalog.dropTempView("people")

True

In [30]:
# Check if a temporary view exists
view_exists = spark.catalog.tableExists("people")
view_exists

False

### Data Frame

In [31]:
emp_data = [
    (1, "John"), (2, "Alice"), (3, "Bob"), (4, "Emily"),
    (5, "David"), (6, "Sarah"), (7, "Michael"), (8, "Lisa"),
    (9, "William")
]

In [32]:
employees = spark.createDataFrame(emp_data, ['id', 'name'])
employees.show()

+---+-------+
| id|   name|
+---+-------+
|  1|   John|
|  2|  Alice|
|  3|    Bob|
|  4|  Emily|
|  5|  David|
|  6|  Sarah|
|  7|Michael|
|  8|   Lisa|
|  9|William|
+---+-------+



In [33]:
emp_salary = [
    ("HR", 1, 60000), ("HR", 2, 55000), ("HR", 3, 58000),
    ("IT", 4, 70000), ("IT", 5, 72000), ("IT", 6, 68000),
    ("Sales", 7, 75000), ("Sales", 8, 78000), ("Sales", 9, 77000)
]

In [35]:
salaries = spark.createDataFrame(emp_salary, ['department', 'id', 'salary'])
salaries.show()

+----------+---+------+
|department| id|salary|
+----------+---+------+
|        HR|  1| 60000|
|        HR|  2| 55000|
|        HR|  3| 58000|
|        IT|  4| 70000|
|        IT|  5| 72000|
|        IT|  6| 68000|
|     Sales|  7| 75000|
|     Sales|  8| 78000|
|     Sales|  9| 77000|
+----------+---+------+



In [37]:
# Register as temporary views
employees.createOrReplaceTempView("employees")
salaries.createOrReplaceTempView("salaries")

In [38]:
# Subquery to find employees with salaries above average
result = spark.sql("""
    SELECT name
    FROM employees
    WHERE id IN (
        SELECT id
        FROM salaries
        WHERE salary > (SELECT AVG(salary) FROM salaries))""")

result.show()

+-------+
|   name|
+-------+
|  Emily|
|  David|
|Michael|
|   Lisa|
|William|
+-------+



In [39]:
# Stop the SparkSession
spark.stop()